In [36]:
! pip install emcee

In [37]:
! git clone https://github.com/fsorrenti/debug.git

Cloning into 'debug'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14 (delta 6), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 10.08 MiB | 13.89 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [38]:
%cd debug

/content/debug/debug


In [39]:
from __future__ import annotations
import sys
import numpy as np
from scipy.stats import multivariate_normal
import emcee


In [40]:
from astropy.coordinates import SkyCoord, spherical_to_cartesian
from astropy import units as u
import math as m
from scipy import integrate

In [41]:
covf = open("Pantheon+SH0ES_STAT+SYS.cov").readlines()
covl = np.array(covf, dtype="float")
shape = covl[0]  # si prende la misura in questo modo, geniale
covmat = covl[1:].reshape((int(shape), int(shape)))

inv=np.linalg.inv(covmat)


filtered_data_file = np.genfromtxt(
    "Pantheon+SH0ES.dat", usecols=(4, 10, 12, 13, 26, 27, 31, 8), skip_header=1
)


z = filtered_data_file[:, 0]
mu = filtered_data_file[:, 1]
ceph_dis = filtered_data_file[:, 2]
is_cal = filtered_data_file[:, 3]
ra = filtered_data_file[:, 4]
dec = filtered_data_file[:, 5]
v_pec = filtered_data_file[:, 6]
m_b = filtered_data_file[:, 7]


In [42]:
def integrand(z, H0, Omat):
    return 1 / (H0 * m.sqrt(Omat * (1 + z) ** 3 + (1 - Omat)))


def dl_monopole(z, H0, Omat):
    c = 299792.458
    return (
        c * (1 + z) * integrate.romberg(integrand, 0, z, args=(H0, Omat))
    )  # return quad(integrand, 0, z, args=(H0, Omat, Olam))[0]

In [43]:
def monopole(z, H0, Omat):
    return 5 * np.log10((dl_monopole(z, H0, Omat))) + 25

In [44]:
def exp_obs_monopole_with_M(
    M, H0, omega_m, mu, z, is_cal, ceph,
):
    number_of_elements = len(mu)
    exp_obs_mon = np.zeros(number_of_elements)

    for i in range(0, number_of_elements):
        if is_cal[i] == 1:
            exp_obs_mon[i] = mu[i] + M - ceph[i]
            # exp_obs_mon[i]=mu[i]-monopole(z6[i],H0,omega_m)
        else:
            exp_obs_mon[i] = mu[i] + M - monopole(z[i], H0, omega_m)

    return exp_obs_mon

In [45]:
def log_likelihood_agnostic(parameters: list[float], inversed_covariance):

    '''the following for the agnostic taylor expansion'''
#    d1, d2, d3, dM  = parameters

#    res=residual_agnostic_taylor_expansion(d1, d2, d3, dM )

  #  z0, d1, d2, d3, dM  = parameters

    Omat,dM  = parameters



    res=exp_obs_monopole_with_M(dM,73.4, Omat, mu, z, is_cal, ceph_dis)


    chi2 = res @ inversed_covariance @ res
    return -0.5 * chi2  # normalization is constant so can be omitted


In [46]:
def log_prior(theta, priors):

    """
    log-prior for the parameters (uniform)
    """
    # theta is the free parameters, priors is a list of two elements
    mu, halfwidth = priors
    for mu_i, theta_i, halfwidth_i in zip(mu, theta, halfwidth):
        if theta_i > mu_i + halfwidth_i or theta_i < mu_i - halfwidth_i:
            return -np.inf
    return 0


In [47]:
def log_probability(parameters: list[float], priors, inversed_covariance):
    """
    log-probability that governs the rate of acceptance for given proposed parameter.
    """
    lp = log_prior(parameters, priors)
    if not np.isfinite(lp):
        return -np.inf

    ll = log_likelihood_agnostic(parameters, inversed_covariance)
    if not np.isfinite(ll):
        return -np.inf

    return lp + ll

In [48]:
def run_MCMC(
    nwalkers,
    number_of_steps,
    parameters_values,
    prior_width,
    outpath=None,
    contd=None,
):
    """
    a wrapper file
    """

    inversed_covariance = inv

    # inversed_covariance=inversed_covariance_data



    if outpath is None:
        outpath = "./results.h5"
    # load data

    # priors
    reference_values = np.array(parameters_values, dtype=float)
    widths = np.array(prior_width)
    priors = [reference_values, widths]

    # initial guess array for each walker:
    x0 = np.random.uniform(
        reference_values - widths,
        reference_values + widths,
        size=(nwalkers, len(reference_values)),
    )
    nwalkers, ndim = x0.shape

    # save file
    backend = emcee.backends.HDFBackend(outpath)
    if contd:  # it is true if you want to continue the run
        print(f"initial size: {backend.iteration}", flush=True)
    else:
        backend.reset(nwalkers, ndim)

        # initialize sampler, run MCMC
    sampler = emcee.EnsembleSampler(
        nwalkers,
        ndim,
        log_probability,
        args=[priors, inversed_covariance],
        backend=backend,
    )

    if contd:
        sampler.run_mcmc(
            None, number_of_steps, progress=True, skip_initial_state_check=True
        )
        print(f"final size: {backend.iteration}", flush=True)
    else:
        sampler.run_mcmc(
            x0, number_of_steps, progress=True, skip_initial_state_check=True
        )


In [49]:
# default output file name
OUTPATH = "test_scratch.h5"

# MCMC sampler settings
N_WALKERS = 32  # number of walkers
N_CHAIN = 1000  # length of chain for each walker
CONTD = False  # set CONTD = True to resume sampling (for specific OUTPATH)
# note: keeping CONTD = False may overwrite an existing file.

PARAMETERS_VALUES = [0.5, 0]#per la mia scelta di angoli il coseno puo' essere solo fra 0 e 1
PRIOR_WIDTH = [0.5, 0.5]


In [ ]:
if __name__ == "__main__":  # it just execute the function
    # load least-square, bets-fit results by default
    #
    # note:
    # Passing manually defined arrays to 'lstsq_results' argument
    # and setting PRIOR_WIDTH_RATIO = 0 will allow users to define
    # the desired central values and width of the prior manually.
    # If you wish to use different (e.g., Gaussian) priors, define
    # the analytic Gaussian function in MCMC_utils.py file accordingly.
    # q_lstsq, sigma_lstsq = np.loadtxt(lstsq_results_path, unpack=True)

    # run MCMC
    run_MCMC(
        nwalkers=N_WALKERS,
        number_of_steps=N_CHAIN,
        parameters_values=PARAMETERS_VALUES,
        prior_width=PRIOR_WIDTH,
        outpath=OUTPATH,
        contd=CONTD,
    )


<ipython-input-42-6fc222fc8025>:8: DeprecationWarning: `scipy.integrate.romberg` is deprecated as of SciPy 1.12.0and will be removed in SciPy 1.15.0. Please use`scipy.integrate.quad` instead.
  c * (1 + z) * integrate.romberg(integrand, 0, z, args=(H0, Omat))
  1%|          | 10/1000 [00:53<1:30:23,  5.48s/it]